In [1]:
try:
    from sklearn.datasets import fetch_openml                               #importing dataset
    mnist = fetch_openml('mnist_784', version=1, cache=True)
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')

x = mnist["data"]
y = mnist["target"]

x_train = x[:60000]
y_train = y[:60000]
x_test = x[60000:]
y_test = y[60000:]

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
import numpy as np

np.random.seed(42)
random_idx = np.random.permutation(60000)               #shuffeling datatset

x_train = x_train.iloc[random_idx]
y_train = y_train.iloc[random_idx]

In [3]:
from sklearn.svm import LinearSVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(x_train, y_train)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [4]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(x_train)
accuracy_score(y_train, y_pred)

0.8656166666666667

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32))
x_test_scaled = scaler.transform(x_test.astype(np.float32))

In [6]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(x_train_scaled, y_train)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [7]:
y_pred = lin_clf.predict(x_train_scaled)
accuracy_score(y_train, y_pred)

0.92025

In [8]:
from sklearn.svm import SVC

svm_clf = SVC(decision_function_shape="ovr", 
              gamma="auto")
svm_clf.fit(x_train_scaled[:10000], y_train[:10000])

SVC(gamma='auto')

In [9]:
y_pred = svm_clf.predict(x_train_scaled)
accuracy_score(y_train, y_pred)

0.9476

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
random_search_cv = RandomizedSearchCV(svm_clf, 
                                      param_distributions, 
                                      n_iter=10, 
                                      verbose=2, 
                                      cv=3)
random_search_cv.fit(x_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....C=8.852316058423087, gamma=0.001766074650481071; total time=   1.2s
[CV] END ....C=8.852316058423087, gamma=0.001766074650481071; total time=   1.3s
[CV] END ....C=8.852316058423087, gamma=0.001766074650481071; total time=   0.8s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453384; total time=   0.7s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453384; total time=   0.7s
[CV] END ...C=1.8271960104746645, gamma=0.006364737055453384; total time=   0.8s
[CV] END ....C=9.875199193765326, gamma=0.051349833451870636; total time=   0.7s
[CV] END ....C=9.875199193765326, gamma=0.051349833451870636; total time=   0.7s
[CV] END ....C=9.875199193765326, gamma=0.051349833451870636; total time=   0.7s
[CV] END ......C=6.59992909281409, gamma=0.05991666578466177; total time=   0.7s
[CV] END ......C=6.59992909281409, gamma=0.05991666578466177; total time=   0.7s
[CV] END ......C=6.59992909281409, gamma=0.05991

RandomizedSearchCV(cv=3, estimator=SVC(gamma='auto'),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001FE8A260610>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001FE8A263390>},
                   verbose=2)

In [12]:
random_search_cv.best_estimator_

SVC(C=8.852316058423087, gamma=0.001766074650481071)

In [13]:
random_search_cv.best_estimator_.fit(x_train_scaled, y_train)

SVC(C=8.852316058423087, gamma=0.001766074650481071)

In [14]:
y_pred = random_search_cv.best_estimator_.predict(x_train_scaled)
accuracy_score(y_train, y_pred)

0.99965